In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from supabase import create_client
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

# Initialize Supabase connection
load_dotenv()
supabase = create_client(
    os.getenv("NEXT_PUBLIC_SUPABASE_URL"),
    os.getenv("NEXT_PUBLIC_SUPABASE_KEY")
)

# Helper functions
def fetch_historical_funding_rates():
    """Fetch historical funding rates from Supabase"""
    response = supabase.table('funding_rates').select('*').execute()
    df = pd.DataFrame(response.data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

def fetch_arbitrage_opportunities():
    """Fetch historical arbitrage opportunities"""
    response = supabase.table('funding_arbitrage_opportunities').select('*').execute()
    df = pd.DataFrame(response.data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

def fetch_analysis_summary():
    """Fetch historical analysis summaries"""
    response = supabase.table('funding_analysis_summary').select('*').execute()
    df = pd.DataFrame(response.data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# 1. Load and Process Data
print("Loading data from Supabase...")
funding_rates_df = fetch_historical_funding_rates()
arbitrage_df = fetch_arbitrage_opportunities()
summary_df = fetch_analysis_summary()

# 2. Basic Statistics
print("\n=== Basic Statistics ===")
stats = {
    'Total Records': len(funding_rates_df),
    'Unique Symbols': funding_rates_df['symbol'].nunique(),
    'Date Range': f"{funding_rates_df['timestamp'].min()} to {funding_rates_df['timestamp'].max()}",
    'Average Funding Rate': f"{funding_rates_df['funding_rate'].mean():.6f}",
    'Median Funding Rate': f"{funding_rates_df['funding_rate'].median():.6f}",
}

for key, value in stats.items():
    print(f"{key}: {value}")

# 3. Time Series Analysis
# Create funding rate trends plot
fig = px.line(funding_rates_df.groupby('timestamp')['funding_rate'].mean().reset_index(), 
              x='timestamp', y='funding_rate',
              title='Average Funding Rate Over Time')
fig.show()

# 4. Exchange Comparison
exchange_comparison = funding_rates_df.groupby('exchange')[['funding_rate', 'annualized_rate']].agg({
    'funding_rate': ['mean', 'std', 'count'],
    'annualized_rate': ['mean', 'std']
}).round(6)

print("\n=== Exchange Comparison ===")
print(exchange_comparison)

# 5. Arbitrage Opportunity Analysis
if not arbitrage_df.empty:
    print("\n=== Arbitrage Opportunities Analysis ===")
    arb_stats = {
        'Total Opportunities': len(arbitrage_df),
        'Average Spread': f"{arbitrage_df['spread'].mean():.6f}",
        'Most Common Symbol': arbitrage_df['symbol'].mode().iloc[0],
        'High Confidence Ops': len(arbitrage_df[arbitrage_df['confidence'] == 'High'])
    }
    
    for key, value in arb_stats.items():
        print(f"{key}: {value}")

    # Plot spread distribution
    fig = px.histogram(arbitrage_df, x='spread',
                      title='Distribution of Arbitrage Spreads',
                      nbins=50)
    fig.show()

# 6. Symbol Analysis
symbol_analysis = funding_rates_df.groupby('symbol').agg({
    'funding_rate': ['mean', 'std', 'count'],
    'annualized_rate': 'mean'
}).round(6)

top_symbols = symbol_analysis.nlargest(10, ('funding_rate', 'mean'))
print("\n=== Top 10 Symbols by Average Funding Rate ===")
print(top_symbols)

# 7. Volatility Analysis
funding_rates_df['funding_volatility'] = funding_rates_df.groupby('symbol')['funding_rate'].transform('std')
volatile_symbols = funding_rates_df.groupby('symbol')['funding_volatility'].mean().nlargest(10)

print("\n=== Most Volatile Funding Rates ===")
print(volatile_symbols)

# 8. Correlation Analysis
pivot_rates = funding_rates_df.pivot(index='timestamp', 
                                   columns='symbol', 
                                   values='funding_rate').fillna(method='ffill')
correlation_matrix = pivot_rates.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix.iloc[:10, :10], annot=True, cmap='coolwarm', center=0)
plt.title('Funding Rate Correlations (Top 10 Symbols)')
plt.show()

# 9. Opportunity Score Calculation
def calculate_opportunity_score(row):
    """Calculate opportunity score based on multiple factors"""
    try:
        # Base score from rate magnitude
        score = abs(row['funding_rate']) * 100
        
        # Adjust for prediction accuracy if available
        if 'predicted_rate' in row.index:
            prediction_diff = abs(row['funding_rate'] - row['predicted_rate'])
            score *= (1 + prediction_diff)
            
        # Adjust for volume if available
        if 'mark_price' in row.index:
            price_factor = min(1.0, np.log10(row['mark_price']) / 4)
            score *= (1 + price_factor)
            
        return round(score, 2)
    except Exception as e:
        print(f"Error calculating opportunity score: {e}")
        return 0.0

funding_rates_df['opportunity_score'] = funding_rates_df.apply(calculate_opportunity_score, axis=1)

# Display top opportunities
print("\n=== Current Top Opportunities ===")
latest_opportunities = funding_rates_df[
    funding_rates_df['timestamp'] == funding_rates_df['timestamp'].max()
].nlargest(10, 'opportunity_score')

print(latest_opportunities[['symbol', 'exchange', 'funding_rate', 'opportunity_score']])

# Save analysis results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
analysis_results = {
    'timestamp': timestamp,
    'basic_stats': stats,
    'arbitrage_stats': arb_stats if not arbitrage_df.empty else {},
    'top_opportunities': latest_opportunities.to_dict('records')
}

# Export to JSON for later reference
import json
with open(f'funding_analysis_{timestamp}.json', 'w') as f:
    json.dump(analysis_results, f, indent=2, default=str)

print(f"\nAnalysis complete. Results saved to funding_analysis_{timestamp}.json")

Loading data from Supabase...


APIError: {'code': '42P01', 'details': None, 'hint': None, 'message': 'relation "public.funding_rates" does not exist'}